# Example: Evaluation of Model DQLearning_L08-R12-L1

## 1. Imports

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
module_path = str(os.getcwd())+'\\out\\'

module_path = str(os.path.dirname(os.getcwd()))+'\\output\\'

from env import roroDeck
from agent import sarsa, tdq, dqn
from analysis import *
from analysis.algorithms import *
from analysis import evaluator as evm

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
from datetime import datetime


sns.set(style="whitegrid")
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 11})
plt.rcParams.update({'text.color' : "black",
                     'axes.labelcolor' : "black"})
plt.tight_layout()

## 2. Preparation
Construct a set with **400** (`n_evaluations`) unique **random** stowage plan evaluations.
These evaluations are based on randomly generated stowage plans by the RORO-deck environment.

If the evaluation of a randomly generated stowage plan is equivalent to another evaluation within the set this stowage plan is discarded.
If the this size is not reached within **50_000** (`time_out`) iterations than this procedure is stopped.

In [ ]:
n_evaluations = 400
time_out = 50_000
start=datetime.now()

Load a model and print the training parameter.

In [ ]:
loading_list_1 =  np.array([[ 0,  1,  2,  3,  4,  5,  6],
                           [ 5,  5, -1, -1,  2,  2,  2],
                           [ 1,  1,  0,  0,  1,  1,  1],
                           [ 1,  2,  1,  2,  2,  1,  2],
                           [ 2,  3,  2,  3,  2,  2,  3],
                           [ 0,  0,  0,  0,  1,  0,  0]])

loading_list_2 = np.array([[0, 1, 2, 3, 4], 
                         [5, 5, -1, -1, 2], 
                         [1, 1, 0, 0, 1], 
                         [1, 2, 1, 2, 2], 
                         [3, 4, 2, 3, 2], 
                         [0, 0, 0, 0, 1]]) 

# Note that loading_list_0 is the default
env = roroDeck.RoRoDeck(lanes=8, rows=12, stochastic=True, vehicle_data=loading_list_1)
evaluator = evm.Evaluator(env.vehicle_data, env.grid)

agent = dqn.DQLearningAgent(env, module_path)

agent.load_model(module_path+"DQLearning\\L08-R12-L1\\DQLearning_L08-R12-L1.h5")

`get_sorted_random_stowage_plans()` constructs the random set:

In [ ]:
def get_sorted_random_stowage_plans():
    random_stowage_plans = set()
    i = 0
    
    while len(random_stowage_plans) < n_evaluations and i < time_out:
        done = False
        env.reset()
        while not done:
            state, reward, done, info = env.step(env.action_space_sample())
        evaluation = evaluator.evaluate(env.get_stowage_plan())
        random_stowage_plans.add(evaluation)
        i+=1
        if i%500 == 0:
            print(str(i)
                  + ' of {}\t unique stowage plan evaluations:\t'.format(time_out)
                  + str(len(random_stowage_plans)))
    if i == time_out:
        print('\n\nWARNING:\tCould not construct {} evaluations.'.format(n_evaluations))
        print('\t\tActual number is {}'.format(len(random_stowage_plans)))

    random_stowage_plans = list(random_stowage_plans)
    random_stowage_plans.sort()
    return random_stowage_plans

`valuate model()` evaluates ranks. Three metrices may be calculated by this:

- deterministic rank ie. the stowage plan if the environment is set to deterministic
- avg. ranks 95% and 99%
*(Construct **50** stowage plans with the model and some randomness, report mean and standard deviation)*
- lowest ranks from the avg. ranks 95% and 99%


In [ ]:
def valuate_model(env_local, plans, n=50, info=False):
    performance = []
    for i in range(n):
        env_local.reset()
        agent.execute()
        evaluation = evaluator.evaluate(env_local.get_stowage_plan())
        plans_val = plans.copy()
        plans_val.append(evaluation)
        plans_val = list(dict.fromkeys(plans_val))
        
        plans_val.sort()
       
        if len(plans_val) == len(plans)+1:
            plans_val = plans_val[1:]

        
        #at which postion is the stowage plan of the agent. (maximal performance 100%)
        for ix,i in enumerate(plans_val):
            if i == evaluation:
                if info:
                    print(str(ix+1)+". Position of "+str(len(plans_val))+ \
                      "\t Performance of model: "+str((ix+1)/(len(plans_val))))
                performance += [(ix+1)/(len(plans_val))]
                break
        
    return np.array(performance)

## 2. Evaluations

### 2.1 Create Random Stowage Plans

In [ ]:
random_plans = get_sorted_random_stowage_plans()

### 2.2 Rank Deterministic

In [ ]:
env.p = 1.
performance = valuate_model(env,random_plans, n=1, info=False)
print('Rank deterministic: ',performance[0])

### 2.3 Rank with p=99%

In [ ]:
env.p = 0.99
performance99 = valuate_model(env,random_plans, n=100, info=False)
print('Rank at 99%:\nMean:\t',performance99.mean(),'\nStd.:\t',performance99.std())

### 2.4 Rank with p=95%

In [ ]:
env.p = 0.95
performance95 = valuate_model(env,random_plans, n=100, info=False)
print('Rank at 95%:\nMean:\t',performance95.mean(),'\nStd.:\t',performance95.std())

### 2.5 Minimal Rank with p=99% and p=95%

In [ ]:
print('Lowest Rank\nat 99%:\t',performance99.min(),'\nat 95%:\t',performance95.min())

### 2.6 Critical p with Mann-Whitney-U

In [ ]:
from scipy.stats import ranksums
from scipy.stats import mannwhitneyu

Reduce `env.p` from 90% to 0% and do the Mann-Whitney-U test. Stop when critical p is reached

In [ ]:
a = np.linspace(0.98,0,71)

In [ ]:
random = random_plans.copy()
random_ranks_prev, agent_ranks_prev = None,None
for ix, temperature in enumerate(a):
    
    
    
    print('Process {}%\t\tCurrent env.p={}'.format((round(ix*100/len(a),2)),(round(temperature,4))))

    env.stochastic= True
    env.p = temperature
    best_sp = []

    while len(best_sp)<100:
        env.reset()
        agent.execute()
        evaluation = evaluator.evaluate(env.get_stowage_plan())
        if evaluation not in random:
            best_sp += [evaluation]
            
    all_ranks = random + best_sp
    all_ranks.sort()
    
    random_ranks = []
    agent_ranks = []
    for rank, i in enumerate(all_ranks):
        if i in best_sp:
            agent_ranks+=[rank]
        else:
            random_ranks+=[rank]
        
    agent_ranks = np.array(agent_ranks)
    random_ranks = np.array(random_ranks)
    
    _,p = mannwhitneyu(random_ranks, agent_ranks, alternative='two-sided')
    print('\t\t\tMann-Whitney-U p_value:',str(round(p,9)))
    if p > 0.01:
        print('STOP -> critical p identified\n')
        break
    random_ranks_prev = random_ranks
    agent_ranks_prev = agent_ranks
    
    
_,p = mannwhitneyu(random_ranks, agent_ranks, alternative='two-sided')
print('The test was not significant anymore at env.p=',temperature)
print(mannwhitneyu(random_ranks, agent_ranks, alternative='two-sided'))
print('The p-value is higher than 1%')
print('\nMann-Whitney-U test statistic of last significant p')
print(mannwhitneyu(random_ranks_prev, agent_ranks_prev, alternative='two-sided'))
print('Statistic of the Wilcoxcon-Ranksum Test:')
print(ranksums(random_ranks_prev,agent_ranks_prev))
print('*'*40)
print('Critical p:\t:',a[ix-1])


## 3. Visualisations

In [ ]:
module_path = str(os.getcwd())+'\\out\\'
os.makedirs(module_path, exist_ok=True)


sns.set(style="whitegrid")
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 11})
plt.rcParams.update({'text.color' : "black",
                     'axes.labelcolor' : "black"})
plt.tight_layout()

### 3.1 Rank Mean over env.p 
Visualise the mean ranks for different level of env.p

In [ ]:
means = []
std = []
x = np.linspace(0,1,21)
for ix,i in enumerate(x):
    print('Process {}%'.format(round(ix*100/len(x),4)))
    env.p = i
    performance = valuate_model(env,random_plans, n=50)
    means += [performance.mean()]
    std += [performance.std()]

fig = plt.figure(figsize=(5.1, 2.2))

plt.plot(x, means)
plt.xlabel("env.p")
plt.ylabel("Rank (Mean of n=50)")
plt.show()

### 3.2 Rank distribution at critical env.p
Visualise critical p Rank distribution

In [ ]:
bw = 50

fig = plt.figure(figsize=(5.1, 2.2))

fig.tight_layout()
gs = fig.add_gridspec(4, 3)

fi_ax1 = fig.add_subplot(gs[0:3, :])
fi_ax2 = fig.add_subplot(gs[3, :],sharex=fi_ax1)


plt.setp(fi_ax1.get_xticklabels(), visible=False)
plt.setp(fi_ax2.get_xticklabels(), visible=True)


ax = sns.kdeplot(np.array(agent_ranks), bw=bw, clip=[0,len(all_ranks)],kernel='epa', ax=fi_ax1,label='Agent')
ax = sns.kdeplot(np.array(random_ranks),ax=ax , bw=bw, clip=[0,len(all_ranks)],kernel='epa',label='Random')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

ax.legend(loc='center left', bbox_to_anchor=(1, box.y0 +0.5* box.height), fontsize=11*(1/0.9))

ax = sns.stripplot(np.array(agent_ranks),alpha=0.5, ax = fi_ax2,label='Agent')
ax = sns.stripplot(np.array(random_ranks),alpha=0.35, ax = ax, color=sns.color_palette("deep")[1],label='Random')
ax.set_xlim(-bw, len(all_ranks)+bw)

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])


    
fi_ax1.set(ylabel='Density')
fi_ax2.set(xlabel='Ranks')

plt.savefig(module_path + '\\denisity_p'+str(round(temperature,4))+'prozent_bw50.pdf', dpi=600, bbox_inches="tight")

In [ ]:
print('This Notebook took:',datetime.now()-start,'s')